

<div style=" text-align: center">

<div style="margin-bottom: -20px">
    <img style="display: inline; width: 200px" src="https://www.saashub.com/images/app/service_logos/239/26gdror9vj3e/medium.png?1679110815" />
&nbsp;&nbsp;  <span style="font-weight: 100">x</span>  &nbsp;&nbsp;&nbsp;&nbsp;
    <img style="display: inline; width: 200px" src="https://raw.githubusercontent.com/PySport/kloppy/master/docs/logo.png" />
        </div>
<!--     <h1>Kloppy <span style="font-weight: 100">x</span> DuckDB</h1> -->
    <br />
        <h6 style="width: 800px; margin: auto">Unlocking Valuable Sports Insights: Analyzing SportsCode XML Files with DuckDB and Kloppy</h6>
    <br />
    <span style="font-size: 20px">Koen Vossen - PySport Meetup Eindhoven - April 2023</span>
    </div>






## Content

- Introduction
- DuckDB
- Kloppy
- Code
- Conclusion

# Introduction

- founder TeamTV
- korfbal coach
- founder PySport

# DuckDB

DuckDB is an in-process SQL OLAP database management system

<img src="duckdb.png" style="width: 50%; margin: auto" />

# In-process database vs External database

<img src="duckdb2.png?t=3" style="width: 90%; margin: auto" />

# In-process database: SQLite vs DuckDB

<img src="sqlite-duckdb.png" style="width: 90%; margin: auto" />

# Kloppy

Standardizing soccer tracking and event data

#### Loading data
- Load **public datasets** to get started right away
- Understandable **standardized data models** for tracking and event data
- Out-of-the-box **(de)serializing** tracking and event data from different vendors into standardized models and vice versa

#### Processing data
- Flexibly transform a dataset's **pitch dimensions** from one format to another (e.g., from OPTA's 100x100 to TRACAB meters)
- Transform the **orientation** of a dataset (e.g., from TRACAB fixed orientation to "Home Team" orientation)

<img src="data-logger.jpeg" style="width: 80%; margin: auto" />

<img src="what-kloppy-does.png" style="width: 100%; margin: auto" />

<img src="shapes.png" style="width: 100%; margin: auto" />

# Code



In [77]:
%pip install kloppy

Note: you may need to restart the kernel to use updated packages.


In [78]:
# Import sportscode deserializer
from kloppy import sportscode

In [79]:
# Load the XML file into a kloppy dataset
dataset = sportscode.load("files/DBO - OR 2 (1).xml")


In [80]:
# Show all records within the dataset
for record in dataset:
    print(f"{record.code} - {record.period.id} - {record.timestamp}")

Sander - 1 - 98.03999999999999
Start Q - 1 - 101.02000000000001
Start - 1 - 100.03517986699994
Joep - 1 - 154.73000000000002
Duel - - 1 - 159.88
Gijs - 1 - 167.07999999999998
David - 1 - 169.39
Duel + - 1 - 171.55
Sander - 1 - 185.12
Gijs - 1 - 193.55
Passing + - 1 - 199.14
Jamie - 1 - 213.32
Special Robert - 1 - 249.87
Gijs - 1 - 255.37
Sander - 1 - 256.8
Duel + - 1 - 260.02
Jelle - 1 - 262.36
Thomas - 1 - 282.93
Contact Lead + - 1 - 285.95
Jamie - 1 - 290.08000000000004
Contact Lead - - 1 - 295.58000000000004
Sheldon - 1 - 298.66999999999996
Jamie - 1 - 317.35
Circle Possession + - 1 - 317.84
Phil - 1 - 319.55
Jamie - 1 - 322.94
Joep - 1 - 325.28
Gijs - 1 - 348.84
Gijs - 1 - 356.96
Jamie - 1 - 357.74
Pirmin - 1 - 369.59
Phil - 1 - 375.98
Jelle - 1 - 378.46
Passing + - 1 - 380.73
Laurens - 1 - 385.23
Thomas - 1 - 387.0
Circle Possession + - 1 - 387.07
Jelle - 1 - 390.95
Special Robert - 1 - 442.64
Sander - 1 - 459.14
Duel - - 1 - 464.3
David - 1 - 466.72
Duel + - 1 - 472.97
Pirmin - 1

# Lets do some pre processing

In [81]:
from kloppy.domain import Period
dataset = sportscode.load("files/DBO - OR 2 (1).xml")

period = None
for record in dataset.records:
    if record.code == 'Start':
        if period:
            # End current period
            period.end_timestamp = record.timestamp - 0.001
            
        period = Period(
            id=period.id + 1 if period else 1,
            start_timestamp=record.timestamp,
            end_timestamp=None
        )
        
    if not period:
        # Data before first 'Period marker'
        continue

    record.period = period
    record.timestamp -= period.start_timestamp

In [82]:
for record in dataset:
    print(f"{record.code} - {record.period.id} - {record.timestamp}")

Sander - 1 - 98.03999999999999
Start Q - 1 - 101.02000000000001
Start - 1 - 0.0
Joep - 1 - 54.69482013300008
Duel - - 1 - 59.844820133000056
Gijs - 1 - 67.04482013300004
David - 1 - 69.35482013300005
Duel + - 1 - 71.51482013300007
Sander - 1 - 85.08482013300006
Gijs - 1 - 93.51482013300007
Passing + - 1 - 99.10482013300005
Jamie - 1 - 113.28482013300005
Special Robert - 1 - 149.83482013300005
Gijs - 1 - 155.33482013300005
Sander - 1 - 156.76482013300006
Duel + - 1 - 159.98482013300003
Jelle - 1 - 162.32482013300006
Thomas - 1 - 182.89482013300005
Contact Lead + - 1 - 185.91482013300003
Jamie - 1 - 190.0448201330001
Contact Lead - - 1 - 195.5448201330001
Sheldon - 1 - 198.634820133
Jamie - 1 - 217.31482013300007
Circle Possession + - 1 - 217.80482013300002
Phil - 1 - 219.51482013300006
Jamie - 1 - 222.90482013300004
Joep - 1 - 225.24482013300002
Gijs - 1 - 248.80482013300002
Gijs - 1 - 256.924820133
Jamie - 1 - 257.70482013300006
Pirmin - 1 - 269.554820133
Phil - 1 - 275.94482013300006


# Lets do some data cleaning

In [83]:
PLAYERS = [
    "Gijs", "Jamie", "Sander", "Jelle", "Sheldon", "Phil"
]

players_dataset = dataset.filter(lambda item: item.code in PLAYERS)

In [84]:
for record in players_dataset:
    print(f"{record.code} - {record.period.id} - {record.timestamp}")

Sander - 1 - 98.03999999999999
Gijs - 1 - 67.04482013300004
Sander - 1 - 85.08482013300006
Gijs - 1 - 93.51482013300007
Jamie - 1 - 113.28482013300005
Gijs - 1 - 155.33482013300005
Sander - 1 - 156.76482013300006
Jelle - 1 - 162.32482013300006
Jamie - 1 - 190.0448201330001
Sheldon - 1 - 198.634820133
Jamie - 1 - 217.31482013300007
Phil - 1 - 219.51482013300006
Jamie - 1 - 222.90482013300004
Gijs - 1 - 248.80482013300002
Gijs - 1 - 256.924820133
Jamie - 1 - 257.70482013300006
Phil - 1 - 275.94482013300006
Jelle - 1 - 278.424820133
Jelle - 1 - 290.91482013300003
Sander - 1 - 359.10482013300003
Sander - 1 - 507.1548201330001
Jelle - 1 - 531.5348201330002
Phil - 1 - 541.6748201330001
Gijs - 1 - 870.4248201330001
Gijs - 1 - 916.0748201330001
Sheldon - 1 - 939.9448201330001
Sander - 1 - 963.1148201330002
Gijs - 1 - 975.2448201330001
Gijs - 1 - 1049.6348201330002
Jelle - 2 - 2.6621400459998767
Sander - 2 - 13.752140045999795
Jelle - 2 - 29.32214004599996
Jelle - 2 - 52.26214004600001
Sander -

In [85]:
# Convert kloppy Dataset into Pandas Dataframe
players_data = players_dataset.to_df()
players_data

,code_id,period_id,timestamp,end_timestamp,code,Duel +,BW1,Passing +,BV3,Contact Lead -,...,Duel -,Contact Lead +,Passing -,Box +,BV2,Dynamic Midfield +,BW2,Dynamic Midfield -,BW3,BV1
0,1,1,98.040000,112.05,Sander,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,1,67.044820,181.09,Gijs,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,1,85.084820,199.13,Sander,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,1,93.514820,207.55,Gijs,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12,1,113.284820,227.32,Jamie,NaN,NaN,NaN,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,438,4,1007.944435,4766.55,Jamie,NaN,NaN,NaN,NaN,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,440,4,1030.294435,4788.90,Sander,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,441,4,1037.364435,4795.96,Jamie,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,443,4,1090.664435,4849.27,Sheldon,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Lets use DuckDB

In [86]:
# This requires DuckDB 0.7.2 (preview release)
%pip install duckdb==0.7.2.dev1381

Note: you may need to restart the kernel to use updated packages.


In [87]:
import duckdb
con = duckdb.connect()
con

In [88]:
con.query('SELECT timestamp, code, "Duel +", "Duel -" FROM players_data WHERE "Duel +" OR "Duel -" ORDER BY period_id, timestamp LIMIT 10')

┌────────────────────┬─────────┬─────────┬─────────┐
│     timestamp      │  code   │ Duel +  │ Duel -  │
│       double       │ varchar │ boolean │ boolean │
├────────────────────┼─────────┼─────────┼─────────┤
│  67.04482013300004 │ Gijs    │ true    │ NULL    │
│ 155.33482013300005 │ Gijs    │ true    │ NULL    │
│ 156.76482013300006 │ Sander  │ true    │ NULL    │
│ 359.10482013300003 │ Sander  │ NULL    │ true    │
│  531.5348201330002 │ Jelle   │ true    │ NULL    │
│  541.6748201330001 │ Phil    │ true    │ NULL    │
│  916.0748201330001 │ Gijs    │ true    │ NULL    │
│  939.9448201330001 │ Sheldon │ true    │ NULL    │
│ 171.74214004600003 │ Gijs    │ true    │ NULL    │
│  819.9621400459998 │ Sander  │ true    │ NULL    │
├────────────────────┴─────────┴─────────┴─────────┤
│ 10 rows                                4 columns │
└──────────────────────────────────────────────────┘

In [89]:
con.query("""
    SELECT 
        code, period_id, COUNT("Duel +"), COUNT("Duel -") 
    FROM players_data WHERE "Duel +" OR "Duel -" GROUP BY ALL ORDER BY code

""")

┌─────────┬───────────┬─────────────────┬─────────────────┐
│  code   │ period_id │ count("Duel +") │ count("Duel -") │
│ varchar │   int64   │      int64      │      int64      │
├─────────┼───────────┼─────────────────┼─────────────────┤
│ Gijs    │         1 │               3 │               0 │
│ Gijs    │         2 │               1 │               0 │
│ Gijs    │         3 │               5 │               1 │
│ Gijs    │         4 │               0 │               3 │
│ Jamie   │         2 │               1 │               0 │
│ Jamie   │         4 │               1 │               1 │
│ Jelle   │         1 │               1 │               0 │
│ Jelle   │         4 │               2 │               0 │
│ Phil    │         1 │               1 │               0 │
│ Phil    │         3 │               2 │               1 │
│ Phil    │         4 │               1 │               0 │
│ Sander  │         1 │               1 │               1 │
│ Sander  │         2 │               1 

# Some more data cleaning

In [90]:
PLAYERS = [
    "Gijs", "Jamie", "Sander", "Jelle", "Sheldon", "Phil"
]

players_dataset = dataset.filter(lambda item: item.code in PLAYERS)

def onehot_to_cat(labels, mapping):
    for label_name, category in mapping.items():
        if labels.get(label_name):
            return category
    
def duel_encoder(record):
    return onehot_to_cat(
        record.labels,
        {
            "Duel +": "won",
            "Duel -": "lost"
        }
    )
    
players_data_cat = players_dataset.to_df(
    "period_id",
    player=lambda record: record.code,
    duel=duel_encoder
)


In [91]:
players_data_cat

,period_id,player,duel
0,1,Sander,None
1,1,Gijs,won
2,1,Sander,None
3,1,Gijs,None
4,1,Jamie,None
...,...,...,...
137,4,Jamie,lost
138,4,Sander,None
139,4,Jamie,won
140,4,Sheldon,None


In [92]:
con.query("""
    SELECT 
        player, period_id, 
        COUNT(*) FILTER(duel = 'won') as duel_won
    FROM players_data_cat WHERE duel IS NOT NULL GROUP BY ALL ORDER BY player

""")

┌─────────┬───────────┬──────────┐
│ player  │ period_id │ duel_won │
│ varchar │   int64   │  int64   │
├─────────┼───────────┼──────────┤
│ Gijs    │         1 │        3 │
│ Gijs    │         2 │        1 │
│ Gijs    │         3 │        5 │
│ Gijs    │         4 │        0 │
│ Jamie   │         2 │        1 │
│ Jamie   │         4 │        1 │
│ Jelle   │         1 │        1 │
│ Jelle   │         4 │        2 │
│ Phil    │         1 │        1 │
│ Phil    │         3 │        2 │
│ Phil    │         4 │        1 │
│ Sander  │         1 │        1 │
│ Sander  │         2 │        1 │
│ Sander  │         3 │        1 │
│ Sander  │         4 │        2 │
│ Sheldon │         1 │        1 │
│ Sheldon │         3 │        1 │
│ Sheldon │         4 │        0 │
├─────────┴───────────┴──────────┤
│ 18 rows              3 columns │
└────────────────────────────────┘

In [93]:
con.query("""
    SELECT
        
        player, period_id,
        printf('%.1f%%', (COUNT(*) FILTER(duel='won'))::DECIMAL / COUNT(*) * 100) as won_percentage,
    FROM players_data_cat
    WHERE duel IN ('won', 'lost')
    GROUP BY ALL 
""")

┌─────────┬───────────┬────────────────┐
│ player  │ period_id │ won_percentage │
│ varchar │   int64   │    varchar     │
├─────────┼───────────┼────────────────┤
│ Gijs    │         1 │ 100.0%         │
│ Sander  │         1 │ 50.0%          │
│ Jelle   │         1 │ 100.0%         │
│ Phil    │         1 │ 100.0%         │
│ Sheldon │         1 │ 100.0%         │
│ Gijs    │         2 │ 100.0%         │
│ Sander  │         2 │ 50.0%          │
│ Jamie   │         2 │ 100.0%         │
│ Gijs    │         3 │ 83.3%          │
│ Sander  │         3 │ 100.0%         │
│ Sheldon │         3 │ 50.0%          │
│ Phil    │         3 │ 66.7%          │
│ Jelle   │         4 │ 100.0%         │
│ Sander  │         4 │ 66.7%          │
│ Phil    │         4 │ 100.0%         │
│ Jamie   │         4 │ 50.0%          │
│ Sheldon │         4 │ 0.0%           │
│ Gijs    │         4 │ 0.0%           │
├─────────┴───────────┴────────────────┤
│ 18 rows                    3 columns │
└───────────────

In [94]:
con.query("""
WITH players AS (
    SELECT
        player, 
        period_id,
        printf('%.1f%%', (COUNT(*) FILTER(duel='won'))::DECIMAL / COUNT(*) * 100) as duel_percentage
    FROM players_data_cat 
    WHERE duel IN ('won', 'lost')
    GROUP BY ALL
)
PIVOT 
    players
ON period_id
USING FIRST(duel_percentage)
GROUP BY player
""")

┌─────────┬─────────┬─────────┬─────────┬─────────┐
│ player  │    1    │    2    │    3    │    4    │
│ varchar │ varchar │ varchar │ varchar │ varchar │
├─────────┼─────────┼─────────┼─────────┼─────────┤
│ Gijs    │ 100.0%  │ 100.0%  │ 83.3%   │ 0.0%    │
│ Sander  │ 50.0%   │ 50.0%   │ 100.0%  │ 66.7%   │
│ Jelle   │ 100.0%  │ NULL    │ NULL    │ 100.0%  │
│ Phil    │ 100.0%  │ NULL    │ 66.7%   │ 100.0%  │
│ Sheldon │ 100.0%  │ NULL    │ 50.0%   │ 0.0%    │
│ Jamie   │ NULL    │ 100.0%  │ NULL    │ 50.0%   │
└─────────┴─────────┴─────────┴─────────┴─────────┘

# Read from multiple matches

In [95]:
import os

PLAYERS = [
    "Gijs", "Jamie", "Sander", "Jelle", "Sheldon", "Phil"
]

def onehot_to_cat(labels, mapping):
    for label_name, category in mapping.items():
        if labels.get(label_name):
            return category

        
def duel_encoder(record):
    return onehot_to_cat(
        record.labels,
        {
            "Duel +": "won",
            "Duel -": "lost"
        }
    )


In [96]:

def load_file(filename):
    dataset = sportscode.load(filename)

    period = None
    for record in dataset.records:
        if record.code == 'Start':
            if period:
                # End current period
                period.end_timestamp = record.timestamp - 0.001

            period = Period(
                id=period.id + 1 if period else 1,
                start_timestamp=record.timestamp,
                end_timestamp=None
            )

        if not period:
            # Data before first 'Period marker'
            continue

        record.period = period
        record.timestamp -= period.start_timestamp
        
    players_dataset = dataset.filter(lambda item: item.code in PLAYERS)
    
    match = os.path.basename(filename).split(" 2")[0]
    
    return players_dataset.to_df(
        "period_id", 
        match=match,
        player=lambda record: record.code,
        duel=duel_encoder
    )

In [97]:
import glob

files = glob.glob("files/* 2*.xml")
files

['files/OR - SCHC 2.xml', 'files/DBO - OR 2 (1).xml', 'files/KZ - OR 2.xml']

In [98]:
import pandas as pd
df = pd.concat(
    [load_file(file) for file in files]
)
df

,period_id,match,player,duel
0,1,OR - SCHC,Gijs,None
1,1,OR - SCHC,Phil,None
2,1,OR - SCHC,Jamie,lost
3,1,OR - SCHC,Phil,None
4,1,OR - SCHC,Jelle,None
...,...,...,...,...
131,1,KZ - OR,Jelle,lost
132,1,KZ - OR,Gijs,won
133,1,KZ - OR,Gijs,None
134,1,KZ - OR,Phil,None


In [99]:
con.query("""
SELECT 
    match, 
    period_id, 
    COUNT(*) FILTER (duel='won') as won,
    COUNT(*) FILTER (duel='lost') as lost
FROM df WHERE duel IN ('won', 'lost')
GROUP BY ALL
ORDER BY match, period_id
""")

┌───────────┬───────────┬───────┬───────┐
│   match   │ period_id │  won  │ lost  │
│  varchar  │   int64   │ int64 │ int64 │
├───────────┼───────────┼───────┼───────┤
│ DBO - OR  │         1 │     7 │     1 │
│ DBO - OR  │         2 │     3 │     1 │
│ DBO - OR  │         3 │     9 │     3 │
│ DBO - OR  │         4 │     6 │     7 │
│ KZ - OR   │         1 │    23 │     7 │
│ OR - SCHC │         1 │     1 │     1 │
│ OR - SCHC │         2 │     6 │     0 │
│ OR - SCHC │         3 │     4 │     0 │
│ OR - SCHC │         4 │     6 │     0 │
└───────────┴───────────┴───────┴───────┘

# Using football event data

In [100]:
from kloppy import statsbomb

dataset = statsbomb.load_open_data()

df = (
    dataset
    .filter("pass")
    .to_df(
        "team",
        "player",
        "result",
    )
)

/Users/koen/Developer/Projects/PySport/kloppy/.venv/lib/python3.10/site-packages/kloppy-3.9.0-py3.10.egg/kloppy/_providers/statsbomb.py:67: UserWarning: 

You are about to use StatsBomb public data.
By using this data, you are agreeing to the user agreement. 
The user agreement can be found here: https://github.com/statsbomb/open-data/blob/master/LICENSE.pdf

  warnings.warn(


In [101]:
df

,team,player,result
0,Deportivo Alavés,Jonathan Rodríguez Menéndez,COMPLETE
1,Deportivo Alavés,Guillermo Alfonso Maripán Loaysa,INCOMPLETE
2,Barcelona,Sergio Busquets i Burgos,COMPLETE
3,Barcelona,Ivan Rakitić,COMPLETE
4,Barcelona,Ousmane Dembélé,COMPLETE
...,...,...,...
1127,Barcelona,Sergi Roberto Carnicer,COMPLETE
1128,Barcelona,Arturo Erasmo Vidal Pardo,COMPLETE
1129,Barcelona,Ivan Rakitić,COMPLETE
1130,Barcelona,Jordi Alba Ramos,COMPLETE


In [102]:
con.query("""
SELECT player, result, COUNT(*) FROM 'teamtv://data.parquet' GROUP BY ALL ORDER BY ALL
""")

┌────────────────────────────────────────┬────────────┬──────────────┐
│                 player                 │   result   │ count_star() │
│                varchar                 │  varchar   │    int64     │
├────────────────────────────────────────┼────────────┼──────────────┤
│ Adrián Marín Gómez                     │ COMPLETE   │            4 │
│ Adrián Marín Gómez                     │ INCOMPLETE │            1 │
│ Adrián Marín Gómez                     │ OUT        │            1 │
│ Arthur Henrique Ramos de Oliveira Melo │ NULL       │            1 │
│ Arthur Henrique Ramos de Oliveira Melo │ COMPLETE   │           16 │
│ Arthur Henrique Ramos de Oliveira Melo │ OFFSIDE    │            1 │
│ Arturo Erasmo Vidal Pardo              │ COMPLETE   │            7 │
│ Borja González Tomás                   │ COMPLETE   │            5 │
│ Borja González Tomás                   │ INCOMPLETE │            1 │
│ Borja González Tomás                   │ OUT        │            1 │
│     

In [103]:
con.query("PIVOT df ON result GROUP BY team")

┌──────────────────┬──────────┬────────────┬─────────┬───────┐
│       team       │ COMPLETE │ INCOMPLETE │ OFFSIDE │  OUT  │
│     varchar      │  int64   │   int64    │  int64  │ int64 │
├──────────────────┼──────────┼────────────┼─────────┼───────┤
│ Deportivo Alavés │      161 │         65 │       0 │    15 │
│ Barcelona        │      792 │         85 │       4 │     5 │
└──────────────────┴──────────┴────────────┴─────────┴───────┘

In [104]:
df_shot = (
    dataset
    .filter("shot")
    .to_df(
        "team",
        "player",
        "result",
    )
)
results = con.query("PIVOT df_shot ON result GROUP BY team, player")
results

┌──────────────────┬────────────────────────────────┬─────────┬───────┬────────────┬───────┬───────┐
│       team       │             player             │ BLOCKED │ GOAL  │ OFF_TARGET │ POST  │ SAVED │
│     varchar      │            varchar             │  int64  │ int64 │   int64    │ int64 │ int64 │
├──────────────────┼────────────────────────────────┼─────────┼───────┼────────────┼───────┼───────┤
│ Barcelona        │ Lionel Andrés Messi Cuccittini │       0 │     2 │          2 │     2 │     2 │
│ Barcelona        │ Jordi Alba Ramos               │       1 │     0 │          1 │     0 │     1 │
│ Deportivo Alavés │ Rubén Sobrino Pozuelo          │       0 │     0 │          1 │     0 │     0 │
│ Barcelona        │ Luis Alberto Suárez Díaz       │       1 │     0 │          2 │     0 │     1 │
│ Barcelona        │ Ousmane Dembélé                │       0 │     0 │          2 │     0 │     1 │
│ Barcelona        │ Ivan Rakitić                   │       1 │     0 │          1 │     0 

# Export to excel 😎

In [105]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [106]:
results.df().to_excel("output.xlsx", index=False)

In [107]:
import subprocess 
subprocess.call(["open", "."])

0

# Conclusion

- kloppy can be used to read sportscode xml files
- data cleaning/transformation can be done in a few lines of code
- cleaned kloppy dataset can be transformed into pandas DataFrame
- DuckDB can query pandas DataFrame directly

# Questions?